In [1]:
!pip install seaborn
!pip install h3
!pip install folium
!pip install shapely
!pip install colour


In [2]:
import pathlib
import pandas as pd
import numpy as np

import pandas as pd
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from shapely.geometry import Polygon

from tqdm import tqdm

import folium


import h3

In [3]:
data = pd.read_pickle("./dataH3.pkl").reset_index(drop=True)
data

,hrId,name,type,ierId,level,point,state,action,cardId,isNear,...,acceptOperatorStr,lat,lon,h3-4,h3-5,h3-6,h3-7,h3-8,h3-9,h3-10
0,янв21_СлучНабор_070,Случайный набор номера,Случайный набор номера,[NULL],Обычный,[NULL],Карточка закрыта,[NULL],78cab6b3-8ded-43e1-a1df-ab49009cb500,[NULL],...,[NULL],0.000000,0.000000,84754a9ffffffff,85754e67fffffff,86754e64fffffff,87754e64dffffff,88754e6499fffff,89754e64993ffff,8a754e64992ffff
1,янв14_Молчание_488,Вызов с молчанием,Вызов с молчанием,[NULL],Обычный,"[{""lat"": 48.7458, ""lon"": 44.816}]",Карточка закрыта,[NULL],5c2b8b11-f56a-46cf-a86f-ab4201016101,[NULL],...,[NULL],48.745800,44.816002,842d48bffffffff,852d48affffffff,862d48ad7ffffff,872d48ad2ffffff,882d48ad2bfffff,892d48ad2b3ffff,8a2d48ad2b37fff
2,янв15_Молчание_418,Вызов с молчанием,Вызов с молчанием,[NULL],Обычный,"[{""lat"": 48.4808, ""lon"": 45.046}]",Карточка закрыта,[NULL],7176c1b6-de84-42d7-aa1e-ab4300ce8947,[NULL],...,[NULL],48.480801,45.046001,842d483ffffffff,852d483bfffffff,862d4839fffffff,872d4839bffffff,882d4839b9fffff,892d4839b97ffff,8a2d4839b82ffff
3,янв16_Молчание_138,Вызов с молчанием,Вызов с молчанием,[NULL],Обычный,"[{""lat"": 48.7757, ""lon"": 44.8021}]",Карточка закрыта,[NULL],43f3339b-e832-493b-8d46-ab440091c61d,[NULL],...,[NULL],48.775700,44.802101,842d48bffffffff,852d48affffffff,862d48ad7ffffff,872d48ad5ffffff,882d48ad55fffff,892d48ad557ffff,8a2d48ad554ffff
4,янв17_Молчание_145,Вызов с молчанием,Вызов с молчанием,[NULL],Обычный,"[{""lat"": 48.513, ""lon"": 44.5205}]",Карточка закрыта,[NULL],bb28b388-fbfa-42c5-8762-ab4500962c90,[NULL],...,[NULL],48.513000,44.520500,842d481ffffffff,852d4817fffffff,862d48157ffffff,872d48153ffffff,882d48153dfffff,892d48153cbffff,8a2d48153ca7fff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517009,EHM1_дек18_Молчание_342,Вызов с молчанием,Вызов с молчанием,,Обычный,"[{""lat"": 48.715, ""lon"": 44.5258}]",Карточка закрыта,,eaaa1dac-a35c-4732-92de-ab2700b5616f,,...,,48.715000,44.525799,842d48bffffffff,852d48bbfffffff,862d48b9fffffff,872d48b99ffffff,882d48b995fffff,892d48b9953ffff,8a2d48b99507fff
2517010,EHM1_дек17_Оказани_486,Оказание медицинской помощи,Оказание медицинской помощи,,Обычный,"[{""lat"": 48.4395, ""lon"": 44.4376}]",Карточка закрыта,,5979084d-5e62-4a1a-91c5-ab26017488fa,,...,,48.439499,44.437599,842d481ffffffff,852d4813fffffff,862d4811fffffff,872d48118ffffff,882d481183fffff,892d481182fffff,8a2d481182cffff
2517011,EHM1_дек18_Оказани_122,Оказание медицинской помощи,Оказание медицинской помощи,,Обычный,"[{""lat"": 50.1105, ""lon"": 45.4226}]",Все реагирования завершены,,f51d9497-5f4a-4870-be06-ab27009431e8,,...,,50.110500,45.422600,8410b27ffffffff,8510b273fffffff,8610b2737ffffff,8710b2730ffffff,8810b27309fffff,8910b27308bffff,8a10b2730887fff
2517012,EHM1_дек13_Молчание_032,Вызов с молчанием,Вызов с молчанием,,Обычный,"[{""lat"": 50.2237829922196, ""lon"": 43.448587056...",Карточка закрыта,,9e53b607-1fed-40d3-80db-ab22002c929f,,...,,50.223782,43.448586,8410b65ffffffff,8510b657fffffff,8610b650fffffff,8710b6509ffffff,8810b65093fffff,8910b65093bffff,8a10b6509387fff


In [4]:
data['type'].unique()

array(['Случайный набор номера', 'Вызов с молчанием', 'Детские шалости',
       'Получение справок', 'Неправильное понимание экстренной ситуации',
       'Автоматические ложные вызовы',
       'Прерывание вызова звонящим сразу после подключения',
       'Оказание медицинской помощи', 'Правонарушение', 'Вскрытие дверей',
       'ДТП', 'Неправильный набор номера (ошибочный)',
       'Застрявшие граждане в лифте', 'Психически неуравновешенные люди',
       'не задано', 'Теплоснабжение', 'Водоснабжение',
       'Наезд на пешехода', 'Труп', 'Причинение вреда здоровью',
       'Исчезновение граждан', 'Семейный конфликт', 'Помощь населению',
       'Драка', 'Кража', 'Газоснабжение',
       'Ложные вызовы вследствие сбоя в сети связи',
       'Подозрительное транспортное средство',
       'Застрявшие транспортные средства', 'Электроснабжение',
       'Угон транспортного средства', 'ДТП с пострадавшими',
       'Обрыв кабеля', 'Повреждение транспортного средства',
       'Открытые люки', 'Ножев

In [9]:
%%time
for t in tqdm(data['type'].unique()):
    h3_level='h3-8'
    data_count=pd.DataFrame(data[data['type']==t].value_counts([h3_level])).reset_index()
    data_count.columns=['h3','count']
    data_count['count_norm']=(data_count['count']-data_count['count'].min())/(data_count['count'][3:].max()-data_count['count'].min()+1)
    data_count=data_count[data_count['count_norm']>0.001]



    from colour import Color
    white = Color("#00ff00")
    colors = list(white.range_to(Color("#ff1000"),101))
    data_count['color']=''

    for i in data_count.index:
        data_count['color'].loc[i]=str(colors[min(int((data_count['count_norm'].loc[i])*100),100)])






    V=[]
    Heat=[]

    #Создание карты
    hmap = folium.Map(location=[48.7152,44.5269], zoom_start=9)



    for i in data_count.index: 

        polygon_geom = Polygon(h3.h3_to_geo_boundary(data_count['h3'].loc[i], True)[:6])


        fillColor=data_count['color'].loc[i]
        color=data_count['color'].loc[i]
        
        list_h3=data_count[['count','h3']][data_count['h3']==data_count['h3'].loc[i]].to_html(index=False,)+ \
         '<br>'+str(h3.h3_to_geo_boundary(data_count['h3'].loc[i])[0])
        #+\                data[['timeIsoStr']][(data[h3_level]==data_count['h3'].loc[i])&(data['type']==t)][-5:].to_html(index=False,)


        folium.GeoJson(
            polygon_geom,
            style_function=lambda x,fillColor=fillColor, color=color: {
                "fillColor": fillColor,
                "color": "#000000",
                'weight' :0.3,
                'fillOpacity': 0.5 
            },
        ).add_child(folium.Popup(list_h3, min_width=500, max_width=500)).add_to(hmap)

    
    #for i in data.index: 
    #    popup_text = "{}".format(
    #                      data['count'].loc[i]
    #                      )


    #display(hmap)    
    hmap.save('./map/'+str(h3_level)+'_('+t.replace('/', '_')+').html')


  0% 0/171 [00:00<?, ?it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  1% 1/171 [02:10<6:10:29, 130.76s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  1% 2/171 [03:59<5:32:15, 117.96s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

 12% 21/171 [11:23<28:09, 11.27s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 13% 22/171 [11:38<30:08, 12.14s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 13% 23/171 [11:52<31:57, 12.96s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

 25% 42/171 [13:59<06:58,  3.24s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 25% 43/171 [14:02<06:48,  3.19s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 26% 44/171 [14:04<05:42,  2.70s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

 37% 63/171 [14:37<01:49,  1.02s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 37% 64/171 [14:39<02:25,  1.36s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 38% 65/171 [14:40<02:17,  1.30s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

 49% 84/171 [15:02<01:42,  1.18s/it]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 50% 85/171 [15:03<01:20,  1.07it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 50% 86/171 [15:03<01:06,  1.27it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

 65% 111/171 [15:19<00:39,  1.52it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 67% 115/171 [15:21<00:22,  2.44it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 68% 116/171 [15:21<00:21,  2.58it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

 85% 146/171 [15:41<00:11,  2.15it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 86% 147/171 [15:41<00:10,  2.37it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 87% 148/171 [15:43<00:14,  1.60it/s]/root/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

CPU times: user 15min 22s, sys: 30.4 s, total: 15min 53s
Wall time: 15min 52s
